# EasyRetinaFace

Ported from JoshVarty's *face_detection/EasyRetinaFace.py*.

In [ ]:
#default_exp EasyRetinaFace

In [ ]:
#export
import numpy as np
import torch
from kgl_deepfake.pytorch_retinaface.models.retinaface import RetinaFace
from kgl_deepfake.pytorch_retinaface.data import cfg_mnet, cfg_re50
from kgl_deepfake.pytorch_retinaface.test_widerface import load_model
from kgl_deepfake.pytorch_retinaface.layers.functions.prior_box import PriorBox
from kgl_deepfake.pytorch_retinaface.utils.box_utils import decode, decode_landm
from kgl_deepfake.pytorch_retinaface.utils.nms.py_cpu_nms import py_cpu_nms

In [ ]:
#export
class EasyRetinaFace:

    def __init__(self, path='Pytorch_Retinaface/weights/Resnet50_Final.pth', cpu=True):
        self.cfg = cfg_re50
        self.cfg['pretrain'] = False  # Don't download pretrained ImageNet weights
        self.net = RetinaFace(cfg=self.cfg, phase='test')
        # Download weights from Google Drive. See repo for details: https://github.com/biubug6/Pytorch_Retinaface
        self.net = load_model(self.net, path, cpu)
        self.net = self.net.eval()
        self.device = torch.device('cpu' if cpu else 'cuda')
        self.net = self.net.to(self.device)
        

    def detect_on_multiple_frames(self, frames):
        """
        Get detections from multiple frames.

        NOTE: This does not run in parallel, it simply calls self.detect() sequentially
        This is because with the default image size, we can't predict more than 2 images at a time. :(
        """

        detections = []
        for frame in frames:
            current_detections = self.detect(frame)
            detections.append(current_detections)

        return detections

    def detect(self, frame, scale_bbox=1):
        """
        Get the detections for a single frame
        """
        img = np.float32(frame)
        target_size = 1600
        max_size = 2150
        im_shape = img.shape
        im_size_min = np.min(im_shape[0:2])
        im_size_max = np.max(im_shape[0:2])
        resize = float(target_size) / float(im_size_min)

        if np.round(resize * im_size_max) > max_size:
            resize = float(max_size) / float(im_size_max)

        # Only resize if the image is bigger than we can handle
        resize = min(1.0, resize)
        if resize != 1:
            img = cv2.resize(img, None, None, fx=resize, fy=resize, interpolation=cv2.INTER_LINEAR)

        im_height, im_width, _ = img.shape
        scale = torch.Tensor([img.shape[1], img.shape[0], img.shape[1], img.shape[0]])
        img -= (104, 117, 123)
        img = img.transpose(2, 0, 1)
        img = torch.from_numpy(img).unsqueeze(0)
        img = img.to(self.device)
        scale = scale.to(self.device)

        loc, conf, landms = self.net(img)  # forward pass

        priorbox = PriorBox(self.cfg, image_size=(im_height, im_width))
        priors = priorbox.forward()
        priors = priors.to(self.device)
        prior_data = priors.data
        boxes = decode(loc.data.squeeze(0), prior_data, self.cfg['variance'])
        boxes = boxes * scale / resize
        boxes = boxes.cpu().numpy()
        scores = conf.squeeze(0).data.cpu().numpy()[:, 1]
        landms = decode_landm(landms.data.squeeze(0), prior_data, self.cfg['variance'])
        scale1 = torch.Tensor([img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                               img.shape[3], img.shape[2], img.shape[3], img.shape[2],
                               img.shape[3], img.shape[2]])
        scale1 = scale1.to(self.device)
        landms = landms * scale1 / resize
        landms = landms.cpu().numpy()

        # ignore low scores
        DEFAULT_CONFIDENCE_THRESH = 0.9
        inds = np.where(scores > DEFAULT_CONFIDENCE_THRESH)[0]
        boxes = boxes[inds]
        landms = landms[inds]
        scores = scores[inds]

        # keep top-K before NMS
        order = scores.argsort()[::-1]
        # order = scores.argsort()[::-1][:args.top_k]
        boxes = boxes[order]
        landms = landms[order]
        scores = scores[order]

        # do NMS
        DEFAULT_NMS_THRESH = 0.4
        dets = np.hstack((boxes, scores[:, np.newaxis])).astype(np.float32, copy=False)
        keep = py_cpu_nms(dets, DEFAULT_NMS_THRESH)
        # keep = nms(dets, args.nms_threshold,force_cpu=args.cpu)
        dets = dets[keep, :]
        landms = landms[keep]

        # keep top-K faster NMS
        # dets = dets[:args.keep_top_k, :]
        # landms = landms[:args.keep_top_k, :]

        if scale_bbox != 1: 
            dets = self.scale_bbox(dets, (im_height, im_width), scale=scale_bbox)

        return dets

    def scale_bbox(self, dets, image_size, scale=1.):
        '''
        Scale RetinaFace's default bboxes' width and height.
        '''
        x_min, y_min, x_max, y_max = [dets[:,i] for i in range(4)]
        center_y, center_x = (y_max + y_min) / 2, (x_min + x_max) / 2

        width, height = x_max - x_min, y_max - y_min

        height = height * scale   # increase height by 20%
        width = width * scale     # increase width by 20%

        y_min, y_max = center_y - (height / 2), center_y + (height / 2)
        x_min, x_max = center_x - (width / 2), center_x + (width / 2)

        imh, imw = image_size
        y_min, y_max = [np.clip(o, a_min=0, a_max=imh) for o in (y_min, y_max)]
        x_min, x_max = [np.clip(o, a_min=0, a_max=imw) for o in (x_min, x_max)]
        
        for i, o in enumerate([x_min, y_min, x_max, y_max]): dets[:,i] = o
        return dets

In [ ]:
detector = EasyRetinaFace()

Loading pretrained model from Pytorch_Retinaface/weights/Resnet50_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:456


# - fin